# Lesson 2
___
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
___
- Наименование вакансии
- Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
- Ссылку на саму вакансию        
- Сайт откуда собрана вакансия

In [92]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from pprint import pprint
import re

In [107]:
def preporation_parser(vacancy):
    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
    }
    
    link = 'https://hh.ru/search/vacancy' 
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
            
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(parser_hh(item))
    #pprint(vacancy_date)            
    return vacancy_date

In [108]:
def parser_hh(item):
    vacancy_date = {}
    
    # vacancy name
    vacancy_name = item.find('div',{'class':'vacancy-serp-item__info'}).getText().replace(u'\xa0', u' ')
    vacancy_date['name'] = vacancy_name
    
    # company_name
    company_name = item.find('div',{'class':'vacancy-serp-item__meta-info'}).find('a').getText()
    vacancy_date['company_name'] = company_name
    
    # city 
    city = item.find('span',{'data-qa':'vacancy-serp__vacancy-address'}).getText()
    vacancy_date['city'] = city
    
    # salary
    
    salary = item.find('div',{'class':'vacancy-serp-item__sidebar'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = salary[1]
            salary_currency = salary[2]
        elif salary[0] == 'от':
            salary_min = salary[1]
            salary_max = None
            salary_currency = salary[2]
        elif salary[0] == '':
            salary_min = None
            salary_max = None
            salary_currency = None
            salary_currency = None
        else:
            salary_min = salary[0]
            salary_max = salary[1]
            salary_currency = salary[2]
        
        
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    # link
    vacancy_link = item.find('a',{'class':'bloko-link'})['href']
    vacancy_date['link'] = vacancy_link
    
    # site
    vacancy_date['site'] = 'hh.ru'
    
    return vacancy_date

In [ ]:
preporation_parser('python')

In [109]:
def parser_vacancy(vacancy):
    vacancy_date = []
    vacancy_date.extend(preporation_parser(vacancy))
    df = pd.DataFrame(vacancy_date)

    return df

In [110]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [111]:
df.head()

,name,company_name,city,salary_min,salary_max,salary_currency,link,site
0,Скилловый Python Developer на бэкенд,Интеллектуальные системы,"Санкт-Петербург, Петроградская",250000,None,руб.,https://krasnodar.hh.ru/vacancy/37765447?query...,hh.ru
1,Python / Django fullstack программист,ООО Чистый Маркетинг,Москва,110000,180000,руб.,https://krasnodar.hh.ru/vacancy/37461514?query...,hh.ru
2,Программист-разработчик python (midle),CATAPULTO.RU,Москва,100000,180000,руб.,https://krasnodar.hh.ru/vacancy/37674249?query...,hh.ru
3,"Backend разработчик Python (Middle, Senior)",Infatica Pte Ltd,Москва,80000,120000,руб.,https://krasnodar.hh.ru/vacancy/37742512?query...,hh.ru
4,Senior Data Engineer / Python,ООО IHS Global,"Минск, Немига",None,None,None,https://krasnodar.hh.ru/vacancy/37762953?query...,hh.ru
